In [5]:
import requests
from bs4 import BeautifulSoup
import openpyxl
from tqdm import tqdm

# open the excel sheet and get the data
workbook = openpyxl.load_workbook('/Users/rentsher/Desktop/Gumlet/business_domain.xlsx')
sheet = workbook['Sheet7']


# create a list to store the data
data = []

# get the total number of rows in the excel sheet
total_rows = sheet.max_row

# create a progress bar using tqdm
pbar = tqdm(total=total_rows, desc="Fetching data")

# loop through each row in the excel sheet
for row in sheet.iter_rows():
    # get the value of the first column (the url)
    url = row[0].value
    # make a request to the url
    r = requests.get(url)
    # parse the html using beautiful soup
    soup = BeautifulSoup(r.text, 'html.parser')
    # find all the divs with the class 'data'
    divs = soup.find_all('div', class_='StatisticsCategoriesDistribution__CategoryTitleValueWrapper-fnuckk-5 dvxqnd')
    # create a list to store the data for this row
    row_data = []
    # loop through each div
    for div in divs:
        # split the div into a list
        split_data = div.text.split('\n')
        # add each item in the list to the row data
        for item in split_data:
            row_data.append(item)
    # if there is no data, add empty strings to the row data
    if not row_data:
        row_data = [""]*len(row)
    # add the row data to the data list
    data.append(row_data)
    # update the progress bar
    pbar.update(1)

# close the progress bar
pbar.close()

# loop through the data list and update the excel sheet
for i, row in enumerate(data):
    for j, item in enumerate(row):
        # update the cell with the new data
        sheet.cell(row=i+2, column=j+3).value = item

# save the changes to the excel sheet
workbook.save('/Users/rentsher/Desktop/Gumlet/business_domain.xlsx')


Fetching data: 100%|████████████████████████████| 72/72 [01:22<00:00,  1.14s/it]


In [8]:
import openpyxl
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# Load the Excel workbook
wb = openpyxl.load_workbook("/Users/rentsher/Desktop/Gumlet/scrip_test.xlsx")
ws = wb.active

# Get the number of rows in the sheet
num_rows = ws.max_row

# Define the counters for Column C and Column D
counter_c = 0
counter_d = 0

# Define the API endpoint
api_endpoint = "https://api.thecompaniesapi.com/v1/companies/"

# Define the API token
api_token = "aVBK8Iyo"

# Iterate over each row in the sheet and fetch the required data
for row in tqdm(ws.iter_rows(min_row=2, max_row=num_rows, max_col=2), total=num_rows-1):
    # Fetch data using TheCompaniesAPI
    domain = row[0].value
    api_url = api_endpoint + domain + "?token=" + api_token
    response = requests.get(api_url)
    json_data = response.json()
    
    # Update the corresponding cell in Column E for the company's name
    if 'name' in json_data:
        ws.cell(row=row[0].row, column=5).value = json_data['name']
    
    # Update the corresponding cell in Column F for the company's description
    if 'description' in json_data:
        ws.cell(row=row[0].row, column=6).value = json_data['description']
    # Update the corresponding cell in Column G for the company's description
    if 'country' in json_data:
        ws.cell(row=row[0].row, column=7).value = json_data['country']

# Save the updated Excel workbook
wb.save("/Users/rentsher/Desktop/Gumlet/scrip_test.xlsx")


  0%|                                                     | 0/8 [00:00<?, ?it/s]


ConnectionError: HTTPSConnectionPool(host='api.thecompaniesapi.com', port=443): Max retries exceeded with url: /v1/companies/chaturbate.com?token=aVBK8Iyo (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f8a68dbd160>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))